In [1]:
using StructuralCausalModels, Test

┌ Info: Precompiling StructuralCausalModels [a41e6734-49ce-4065-8b83-aff084c01dfd]
└ @ Base loading.jl:1278


In [2]:
ProjDir = scm_path("..")
cd(ProjDir) #do

In [3]:
ProjDir

"/Users/rob/.julia/dev/StructuralCausalModels/"

In [4]:
fname = scm_path("..", "examples", "Shipley", "fig2.6", "fig2.6.dot")
Sys.isapple() && run(`open -a GraphViz.app $(fname)`);

In [5]:
# Read `=>` as `~` in regression models, or `<-` in causal models.
d = OrderedDict(
  :u => [:x, :v],
  :s1 => [:u],
  :w => [:v, :y],
  :s2 => [:w]
);

In [6]:
dag = DAG("fig2.6", d);

In [7]:
dag.a

7×7 Named Array{Int64,2}
Cols ╲ Rows │  :u   :x   :v  :s1   :w   :y  :s2
────────────┼──────────────────────────────────
:u          │   0    1    1    0    0    0    0
:x          │   0    0    0    0    0    0    0
:v          │   0    0    0    0    0    0    0
:s1         │   1    0    0    0    0    0    0
:w          │   0    0    1    0    0    1    0
:y          │   0    0    0    0    0    0    0
:s2         │   0    0    0    0    1    0    0

In [8]:
to_ggm(dag) == "DAG(u ~ x + v, s1 ~ u, w ~ v + y, s2 ~ w)"

true

In [9]:
d_separation(dag, :x, :v)

true

In [15]:
m_separation(dag, :x, :v)

true

In [14]:
@testset "m_separation_02" begin
  
  @test to_ggm(dag) == "DAG(u ~ x + v, s1 ~ u, w ~ v + y, s2 ~ w)"
  @test m_separation(dag, :x, :v; debug=false) == true
  @test m_separation(dag, :x, [:v]; c=:u, debug=false) == false
  @test m_separation(dag, [:x], :v; c=:s1, debug=false) == false
  @test m_separation(dag, [:u], [:w]; debug=false) == false
  @test m_separation(dag, :u, [:w]; c=:v, debug=false) == true

  @test m_separation(dag, [:x], :y) == true
  @test m_separation(dag, :x, :y; c=[:u, :w], debug=false) == false
  @test m_separation(dag, :x, :y; c=[:s1, :s2], debug=false) == false
  @test m_separation(dag, :x, :y; c=[:u, :v, :w], debug=false) == true
  @test m_separation(dag, :x, [:v]; c=:u, debug=false) == false
  @test m_separation(dag, [:x], :v; c=:s1, debug=false) == false;

end;

Test Summary:   | Pass  Total
m_separation_02 |   12     12
